In [16]:
from dotenv import load_dotenv
import os

from langchain_openai import ChatOpenAI

from langchain_core.messages import AnyMessage, AIMessage, HumanMessage

from typing_extensions import TypedDict, Annotated
import operator

from langgraph.graph import StateGraph

_ = load_dotenv()

In [9]:
chat_llm = ChatOpenAI(
    base_url='https://api.deepseek.com',
    model='deepseek-reasoner',
    api_key=os.getenv('DEEPSEEK_API')
)

chat_llm.invoke('Hello, how are you ?').content

"Hello! I'm doing well, thank you for asking! 😊 I'm here and ready to help you with whatever you need. How are you doing today?"

In [14]:
class GraphState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [ ]:
def llm_call(state: GraphState) -> dict:
    response = chat_llm.invoke(state['messages'])
    return {'message': [response]}

In [19]:
def token_counter(state: GraphState) -> dict:
    last_msg = state['messages'][-1]
    text = last_msg.content
    token_number = len(text.split())
    summary = f'Total token number in the generated answer (word count) is {token_number}'
    return {'messages': [AIMessage(content=summary)]}

In [17]:
builder = StateGraph(GraphState)

In [ ]:
builder.add_node('llm_call', llm_call)
builder.add_node('token_counter', token_counter)

In [22]:
builder.set_entry_point('llm_call')
builder.add_edge('llm_call', 'token_counter')
builder.set_finish_point('token_counter')